<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **LangGraph 101: Building Stateful AI Workflows**


### Installing Required Libraries


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [ ]:
%pip install -q langgraph==0.2.57 langchain-ibm==0.3.10

### Importing Required Libraries


In [ ]:
from langgraph.graph import StateGraph

In [ ]:
from typing import TypedDict, Optional

class AuthState(TypedDict):
    username: Optional[str]
    password: Optional[str]
    is_authenticated: Optional[bool]
    output: Optional[str]

In [ ]:
def input_node(state):
    print(state)
    if state.get('username', "") =="":
        state['username'] = input("What is your username?")

    password = input("Enter your password: ")

    return {"password":password}

In [ ]:
def validate_credentials_node(state):
    # Extract username and password from the state
    username = state.get("username", "")
    password = state.get("password", "")

    print("Username :", username, "Password :", password)
    # Simulated credential validation
    if username == "test_user" and password == "secure_password":
        is_authenticated = True
    else:
        is_authenticated = False

    # Return the updated state with authentication result
    return {"is_authenticated": is_authenticated}

In [ ]:
auth_state_3: AuthState = {
    "username":"test_user",
    "password":  "secure_password",
    "is_authenticated": False,
    "output": "Authentication failed. Please try again."
}
print(f"auth_state_3: {auth_state_3}")

In [ ]:
# Define the success node
def success_node(state):
    return {"output": "Authentication successful! Welcome."}

In [ ]:
# Define the failure node
def failure_node(state):
    return {"output": "Not Successfull, please try again!"}

In [ ]:
def router(state):
    if state['is_authenticated']:
        return "success_node"
    else:
        return "failure_node"

#### Creating the Graph  


In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph import END

# Create an instance of StateGraph with the GraphState structure
workflow = StateGraph(AuthState)
workflow

#### Adding Nodes to the Graph  

In [ ]:
workflow.add_node("InputNode", input_node)

In [ ]:
workflow.add_node("ValidateCredential", validate_credentials_node)

In [ ]:
workflow.add_node("Success", success_node)

In [ ]:
workflow.add_node("Failure", failure_node)

In [ ]:
workflow.add_edge("InputNode", "ValidateCredential")


In [ ]:
workflow.add_edge("Success", END)

In [ ]:
workflow.add_edge("Failure", "InputNode")

In [ ]:
workflow.add_conditional_edges("ValidateCredential", router, {"success_node": "Success", "failure_node": "Failure"})

In [ ]:
workflow.set_entry_point("InputNode")

#### Compiling the Workflow  


In [ ]:
app = workflow.compile()


#### Running the Application  

Once the workflow is compiled, we can run it by invoking the application with the required inputs. The `invoke` method takes an initial state (a dictionary of input values) and starts execution from the entry point defined in the workflow.

<p style='color: red'><b>Note:</b> The correct password is <code>secure_password</code>, so make sure to enter that to authenticate successfully.</p>


In [ ]:
inputs = {"username": "test_user"}
result = app.invoke(inputs)
print(result)

### **Building a QA Workflow Specific to the Guided Project**

In [ ]:
# Define the structure of the QA state
class QAState(TypedDict):
    # 'question' stores the user's input question. It can be a string or None if not provided.
    question: Optional[str]

    # 'context' stores relevant context about the guided project, if the question pertains to it.
    # If the question isn't related to the project, this will be None.
    context: Optional[str]

    # 'answer' stores the generated response or answer. It can be None until the answer is generated.
    answer: Optional[str]

In [ ]:
# Create an example object
qa_state_example = QAState(
    question="What is the purpose of this guided project?",
    context="This project focuses on building a chatbot using Python.",
    answer=None
)

# Print the attributes
for key, value in qa_state_example.items():
    print(f"{key}: {value}")

In [ ]:
def input_validation_node(state):
    # Extract the question from the state, and strip any leading or trailing spaces
    question = state.get("question", "").strip()

    # If the question is empty, return an error message indicating invalid input
    if not question:
        return {"valid": False, "error": "Question cannot be empty."}

    # If the question is valid, return valid status
    return {"valid": True}

In [ ]:
def context_provider_node(state):
    question = state.get("question", "").lower()
    # Check if the question is related to the guided project
    if "langgraph" in question or "guided project" in question:
        context = (
            "This guided project is about using LangGraph, a Python library to design state-based workflows. "
            "LangGraph simplifies building complex applications by connecting modular nodes with conditional edges."
        )
        return {"context": context}
    # If unrelated, set context to null
    return {"context": None}

In [ ]:
from langchain_ibm import ChatWatsonx

llm = ChatWatsonx(
    model_id="ibm/granite-3-3-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
)

In [ ]:
def llm_qa_node(state):
    # Extract the question and context from the state
    question = state.get("question", "")
    context = state.get("context", None)

    # Check for missing context and return a fallback response
    if not context:
        return {"answer": "I don't have enough context to answer your question."}

    # Construct the prompt dynamically
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer the question based on the provided context."

    # Use LangChain's ChatOpenAI to get the response
    try:
        response = llm.invoke(prompt)
        return {"answer": response.content.strip()}
    except Exception as e:
        return {"answer": f"An error occurred: {str(e)}"}

#### **Creating the QA Workflow Graph**  


In [ ]:
qa_workflow = StateGraph(QAState)

In [ ]:
qa_workflow.add_node("InputNode", input_validation_node)

In [ ]:
qa_workflow.add_node("ContextNode", context_provider_node)

In [ ]:
qa_workflow.add_node("QANode", llm_qa_node)

In [ ]:
qa_workflow.set_entry_point("InputNode")

In [ ]:
qa_workflow.add_edge("InputNode", "ContextNode")

In [ ]:
qa_workflow.add_edge("ContextNode", "QANode")

In [ ]:
qa_workflow.add_edge("QANode", END)

In [ ]:
qa_app = qa_workflow.compile()

In [ ]:
qa_app.invoke({"question": "What is the weather today?"})

In [ ]:
qa_app.invoke({"question": "What is LangGraph?"})

In [ ]:
qa_app.invoke({"question": "What is the best guided project?"})

## Exercises

In this exercise, you are going to create a simple counter using LangGraph.


### Exercise 1 - Define the State type

Here, you will define the state schema used by the graph. It should keep track of:
- `n`: a counter starting from 1.
- `letter`: a randomly generated lowercase letter at each step.


In [ ]:
import random
import string
from typing import TypedDict

from langgraph.graph import StateGraph, END

In [ ]:
class ChainState(TypedDict):
  n: int
  letter: str

### Exercise 2 - Create `add()` node Function

This node should represent the `increment` step such that:
- It adds 1 to the current value of n.
- It randomly selects a lowercase letter and updates the letter field.


In [ ]:
def add(state: ChainState) -> ChainState:
  random_letter = random.choice(string.ascii_lowercase)
  return {
    **state,
    "n": state["n"] + 1,
    "letter": random_letter,
  }

### Exercise 3 - Create `print_out()` node Function

This node should print the current state such that:
- It logs the value of n and the current random letter.
- The state is returned.


In [ ]:
def print_out(state: ChainState) -> ChainState:
  print("Current n:", state["n"], "Letter:", state["letter"])
  return state

### Exercise 4 - Stop Condition

Create a function that has a termination condition:
- If the counter reaches 13 or more, the workflow should end.
- Otherwise, it should loop back to add node.


In [ ]:
def stop_condition(state: ChainState) -> bool:
  return state["n"] >= 13

### Exercise 5 - Graph Construction

In this exercise, you'll build the LangGraph flow:

- Create a `StateGraph` object using the `ChainState` that you made.
- Add nodes `add` and `print`.
- Add an edge between `add` and `print`
- Add a conditional edge between `print` and `END` based on `stop_condition`.
- Set `add` as entry point of the graph.


In [ ]:
workflow = StateGraph(ChainState)

workflow.add_node("add", add)
workflow.add_node("print", print_out)

workflow.add_edge("add", "print")
workflow.add_conditional_edges("print", stop_condition, {
    True: END,
    False: "add",
})

workflow.set_entry_point("add")

### Exercise 6 - Compile and Run

Compile the graph and start execution with the given initial input:
- The counter should begin at 1.
- Keep letter empty (to be filled in by the add node).


In [ ]:
app = workflow.compile()

result = app.invoke({"n": 1, "letter": ""})

Copyright © 2024 IBM Corporation. All rights reserved.
